## Import

In [1]:
import requests
from termcolor import colored
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
#target dataframe
DF = pd.DataFrame( columns=["version","year", "km", "price", "pro"])

In [3]:
#handle url function
def _handle_request_result_and_build_soup(page_url):
  request_result = requests.get(page_url)
  if request_result.status_code == 200:
    html_doc =  request_result.text
    soup = BeautifulSoup(html_doc,"html.parser")
    #get_Div(soup)
    #r = requests.get(website_, headers={'Accept': 'application/json'})
    #dftable = pd.read_html(html_doc, attrs = {'class': 'adContainer'})
    return soup

In [4]:
#get all announces
def get_Divs(soup):      
    return soup.findAll("div", {'class' : 'adLineContainer' })
  

## def functions

In [5]:
# create pd series
def getRow(div):
#     print(div.find("h3").findAll("span")[1].text)
#     print(div.find("div", {"class" : "fieldYear"}).text)
#     #print(re.sub('[\s+]', '', div.find("div", {"class" : "fieldMileage"}).text))
#     print(re.match(r"\d+", re.sub('[\s+]', '', div.find("div", {"class" : "fieldMileage"}).text)).group(0))
#     #print(div.find("nobr").text)
#     print(re.match(r"\d+", re.sub('[\s+]', '', div.find("nobr").text)).group(0))
#     print(div.find("div", {"class" : "typeSellerGaranty"}).find("p").text)
        
    try:
        version = div.find("h3").findAll("span")[1].text
        annee = div.find("div", {"class" : "fieldYear"}).text
        km = re.match(r"\d+", re.sub('[\s+]', '', div.find("div", {"class" : "fieldMileage"}).text)).group(0)
        prix = re.match(r"\d+", re.sub('[\s+]', '', div.find("nobr").text)).group(0)
        match = re.match("Professionnel", div.find("div", {"class" : "typeSellerGaranty"}).find("p").text )
        pro = 1 if match else 0
    except AttributeError:
        return None 

    dfDiv = pd.DataFrame([[version, annee, km, prix, pro]], columns=["version","year", "km", "price", "pro"])
    return dfDiv

## build target dataframe

In [6]:
def buildDF(df):
    for i in range(1,20):
        website_ = "https://www.lacentrale.fr/listing?regions=FR-IDF%2CFR-NAQ%2CFR-PAC&makesModelsCommercialNames=RENAULT%3AZOE&page="+str(i)
        page = _handle_request_result_and_build_soup(website_)
        divs = get_Divs(page)
        for div in divs:
            try:
                df = pd.concat([df, getRow(div)], ignore_index=True) 
            except TypeError:
                continue
    return df

In [7]:
DF = buildDF(DF)

In [8]:
DF.head()

,version,year,km,price,pro
0,Q90 ZEN CHARGE RAPIDE,2016,34734,9850,1
1,Q90 INTENS CHARGE RAPIDE,2014,31137,10270,1
2,Q90 ZEN,2014,32564,10360,1
3,Q90 ZEN CHARGE RAPIDE,2016,23984,12080,1
4,Q90 LIFE CHARGE RAPIDE,2015,54661,8010,1


In [9]:
DF.shape

(293, 5)

In [10]:
DF["year"] = pd.to_numeric(DF["year"])
DF["km"] = pd.to_numeric(DF["km"])
DF["price"] = pd.to_numeric(DF["price"])

In [11]:
DF["year"].min()

2012

In [12]:
DF["year"].max()

2019

In [13]:
DF["version"].unique()

array(['Q90 ZEN CHARGE RAPIDE', 'Q90 INTENS CHARGE RAPIDE', 'Q90 ZEN',
       'Q90 LIFE CHARGE RAPIDE', '(2) R90 INTENS', 'Q90 LIFE',
       'INTENS CHARGE RAPIDE', 'LIFE', 'Q90 INTENS', 'INTENS',
       'R110 INTENS', 'R90 ZEN', 'R90 INTENS',
       'Q90 LIFE CHARGE RAPIDE TYPE 2',
       '(2) Q90 LIFE CHARGE RAPIDE GAMME 2017',
       '(2) Q90 INTENS CHARGE RAPIDE GAMME 2017',
       'Q90 INTENS CHARGE RAPIDE TYPE 2', '(2) R90 ZEN',
       'LIFE CHARGE RAPIDE', 'Q90 ZEN CHARGE RAPIDE TYPE 2',
       'R90 INTENS TYPE 2', 'R90 INTENS GAMME 2017', 'INTENS 5P',
       'LIFE R75', 'LIFE CHARGE RAPIDE GAMME 2017 5P', 'R75 LIFE',
       'LIFE CHARGE RAPIDE 5P', '(2) R110 INTENS', 'R90 LIFE TYPE 2',
       'ZEN CHARGE RAPIDE', '(2) R75 LIFE', 'LIFE 5P',
       '(2) Q90 ZEN CHARGE RAPIDE GAMME 2017', '(2) R90 STAR WARS',
       'LIFE CHARGE RAPIDE GAMME 2017', 'ZEN R90 2018', 'INTENS R90',
       'R75 LIFE GAMME 2017', 'INTENS GAMME 2017'], dtype=object)

## build "average rating" dataframe by version and year

In [18]:
DFcote = pd.DataFrame(columns=["version", "year", "mean"])
for version in DF["version"].unique():
    for annee in range(DF["year"].min(), DF["year"].max() + 1):
        site = "https://www.lacentrale.fr/cote-auto-renault-zoe-"+version.lower().replace(" ", "+")+"-"+str(annee)+".html"
        request_result = requests.get(site)
        if request_result.status_code == 200:
            html_doc =  request_result.text
            soup = BeautifulSoup(html_doc,"html.parser")
            try:
                cote = re.sub('[\s+]', '', soup.find("span", {"class" : "jsRefinedQuot"}).text)
                DFcote = pd.concat([DFcote, pd.DataFrame([[version, annee, cote]], columns=["version", "year", "mean"])], ignore_index=True) 
            except AttributeError:
                print(site)
                continue

https://www.lacentrale.fr/cote-auto-renault-zoe-q90+zen+charge+rapide-2018.html
https://www.lacentrale.fr/cote-auto-renault-zoe-q90+zen+charge+rapide-2019.html
https://www.lacentrale.fr/cote-auto-renault-zoe-q90+intens+charge+rapide-2018.html
https://www.lacentrale.fr/cote-auto-renault-zoe-q90+intens+charge+rapide-2019.html
https://www.lacentrale.fr/cote-auto-renault-zoe-q90+zen-2012.html
https://www.lacentrale.fr/cote-auto-renault-zoe-q90+zen-2018.html
https://www.lacentrale.fr/cote-auto-renault-zoe-q90+zen-2019.html
https://www.lacentrale.fr/cote-auto-renault-zoe-q90+life+charge+rapide-2018.html
https://www.lacentrale.fr/cote-auto-renault-zoe-q90+life+charge+rapide-2019.html
https://www.lacentrale.fr/cote-auto-renault-zoe-(2)+r90+intens-2012.html
https://www.lacentrale.fr/cote-auto-renault-zoe-(2)+r90+intens-2013.html
https://www.lacentrale.fr/cote-auto-renault-zoe-(2)+r90+intens-2014.html
https://www.lacentrale.fr/cote-auto-renault-zoe-(2)+r90+intens-2019.html
https://www.lacentrale

https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide+gamme+2017+5p-2016.html
https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide+gamme+2017+5p-2017.html
https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide+gamme+2017+5p-2018.html
https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide+gamme+2017+5p-2019.html
https://www.lacentrale.fr/cote-auto-renault-zoe-r75+life-2012.html
https://www.lacentrale.fr/cote-auto-renault-zoe-r75+life-2013.html
https://www.lacentrale.fr/cote-auto-renault-zoe-r75+life-2014.html
https://www.lacentrale.fr/cote-auto-renault-zoe-r75+life-2015.html
https://www.lacentrale.fr/cote-auto-renault-zoe-r75+life-2016.html
https://www.lacentrale.fr/cote-auto-renault-zoe-r75+life-2017.html
https://www.lacentrale.fr/cote-auto-renault-zoe-r75+life-2018.html
https://www.lacentrale.fr/cote-auto-renault-zoe-r75+life-2019.html
https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide+5p-2012.html
https://www.lacentra

In [19]:
display(DFcote.head())
DFcote.shape

,version,year,mean
0,Q90 ZEN CHARGE RAPIDE,2012,8185
1,Q90 ZEN CHARGE RAPIDE,2013,9203
2,Q90 ZEN CHARGE RAPIDE,2014,11036
3,Q90 ZEN CHARGE RAPIDE,2015,12009
4,Q90 ZEN CHARGE RAPIDE,2016,12174


(109, 3)

## update target df

In [81]:
ratings = []
for i in DF.index:
    #print(DFcote["mean"][DFcote.loc[(DFcote['version'] == DF["version"][i]) & (DFcote['year'] == DF["year"][i])].index].values)
    rating = DFcote["mean"][DFcote.loc[(DFcote['version'] == DF["version"][i]) & (DFcote['year'] == DF["year"][i])].index].values
    
    if not rating.size == 0:
        ratings.append(rating[0])
    else :
        ratings.append(None)
#         print (rating[0])
#         DF.insert(i, "rating", rating[0])

In [82]:
len(ratings)

293

In [83]:
if "rating" in DF.columns:
    DF = DF.drop(["rating"], axis=1)
DF.insert(5, "rating", ratings)

In [84]:
DF.head()

,version,year,km,price,pro,rating
0,Q90 ZEN CHARGE RAPIDE,2016,34734,9850,1,12174
1,Q90 INTENS CHARGE RAPIDE,2014,31137,10270,1,9625
2,Q90 ZEN,2014,32564,10360,1,10809
3,Q90 ZEN CHARGE RAPIDE,2016,23984,12080,1,12174
4,Q90 LIFE CHARGE RAPIDE,2015,54661,8010,1,10646


In [86]:
DF["rating"] = pd.to_numeric(DF["rating"])
DF

,version,year,km,price,pro,rating
0,Q90 ZEN CHARGE RAPIDE,2016,34734,9850,1,12174.0
1,Q90 INTENS CHARGE RAPIDE,2014,31137,10270,1,9625.0
2,Q90 ZEN,2014,32564,10360,1,10809.0
3,Q90 ZEN CHARGE RAPIDE,2016,23984,12080,1,12174.0
4,Q90 LIFE CHARGE RAPIDE,2015,54661,8010,1,10646.0
5,(2) R90 INTENS,2019,6826,18980,1,NaN
6,Q90 LIFE,2016,17282,10190,1,12321.0
7,Q90 LIFE,2014,17705,10960,1,9090.0
8,Q90 LIFE,2016,18995,10030,1,12321.0
9,Q90 LIFE CHARGE RAPIDE,2014,12767,8800,1,10066.0


## covert to csv

In [87]:
DFcote.to_csv("coteMoyenne.csv", ",")
DF.to_csv("zoe.csv", ",")